In [28]:
import pandas as pd
import numpy as np
import charset_normalizer

In [29]:
with open("../../../../../movies.csv", 'rb') as rawdata:
    result = charset_normalizer.detect(rawdata.read(10000))
print(result)

{'encoding': 'utf-8', 'language': 'English', 'confidence': 1.0}


In [30]:
movies = pd.read_csv("../../../../../movies.csv")
movies.info()

# Turn votes column into int
movies['VOTES'] = movies['VOTES'].str.replace(',', '')
movies['VOTES'] = pd.to_numeric(movies['VOTES'], errors='coerce').astype('Int64')
# Make var of all columns that are strings
stringColumns = movies.select_dtypes(include='object').columns


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9999 entries, 0 to 9998
Data columns (total 9 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   MOVIES    9999 non-null   object 
 1   YEAR      9355 non-null   object 
 2   GENRE     9919 non-null   object 
 3   RATING    8179 non-null   float64
 4   ONE-LINE  9999 non-null   object 
 5   STARS     9999 non-null   object 
 6   VOTES     8179 non-null   object 
 7   RunTime   7041 non-null   float64
 8   Gross     460 non-null    object 
dtypes: float64(2), object(7)
memory usage: 703.2+ KB


In [31]:
#Find all newline chars
print('String columns: ', stringColumns)
listSlashN = []
for c in stringColumns:
    listSlashN.append(movies[str(c)].str.startswith('\n').sum())

# Delete them
for c in stringColumns:
    movies[c] = movies[c].str.replace('\n', '')
movies.head()


String columns:  Index(['MOVIES', 'YEAR', 'GENRE', 'ONE-LINE', 'STARS', 'Gross'], dtype='object')


,MOVIES,YEAR,GENRE,RATING,ONE-LINE,STARS,VOTES,RunTime,Gross
0,Blood Red Sky,(2021),"Action, Horror, Thriller",6.1,A woman with a mysterious illness is forced in...,Director:Peter Thorwarth| Stars:Peri B...,21062,121.0,NaN
1,Masters of the Universe: Revelation,(2021- ),"Animation, Action, Adventure",5.0,The war for Eternia begins again in what may b...,"Stars:Chris Wood, Sarah Michel...",17870,25.0,NaN
2,The Walking Dead,(2010-2022),"Drama, Horror, Thriller",8.2,Sheriff Deputy Rick Grimes wakes up from a com...,"Stars:Andrew Lincoln, Norman R...",885805,44.0,NaN
3,Rick and Morty,(2013- ),"Animation, Adventure, Comedy",9.2,An animated series that follows the exploits o...,"Stars:Justin Roiland, Chris Pa...",414849,23.0,NaN
4,Army of Thieves,(2021),"Action, Crime, Horror",NaN,"A prequel, set before the events of Army of th...",Director:Matthias Schweighöfer| Stars:...,<NA>,NaN,NaN


In [32]:
# Remove extraneous spaces
for c in stringColumns:
    movies[c] = movies[c].str.strip()

In [33]:
# Make separate start and end year cols
movies[['START-YEAR', 'END-YEAR']] = movies['YEAR'].str.strip('()').str.split('-', expand=True)
# Make separate actors and directors cols
movies[['DIRECTORS', 'ACTORS']] = movies['STARS'].str.replace(r'Stars:|Director:|Directors:|\s{2,}', '', regex=True).str.split('\|', expand=True)

In [34]:
# count stuff
z = movies['STARS'].str.contains("Director").sum()
y = movies['STARS'].str.contains("\|").sum()
aa = movies['YEAR'].str.startswith('(').sum() + movies['YEAR'].isnull().sum()

aa

9999

In [35]:
# Save changes to data
movies.to_csv('movies-working.csv')

In [10]:
# Subset DataFrame - Only rows with vals for RATING, VOTES, RunTime, Gross
movies_subset = movies.dropna(subset=['RATING', 'VOTES', 'RunTime', 'Gross'])
movies_subset.to_csv('movies-subset.csv')

### Bonus questions
1. What genres appear most in each actor's credits?

In [36]:
# Listify actors and genres cols
movies['GENRE'] = movies['GENRE'].str.split(', ')
movies['ACTORS'] = movies['ACTORS'].str.split(', ')

In [37]:
exploded = movies.explode('GENRE').explode('ACTORS')

unique_genres = exploded['GENRE'].unique()
unique_actors = exploded['ACTORS'].unique()
genre_actors = pd.DataFrame(index=unique_actors, columns=unique_genres)

genre_actors_crosstab = pd.crosstab(exploded['ACTORS'], exploded['GENRE'])

result_df = genre_actors.combine_first(genre_actors_crosstab)
result_df


,Action,Adventure,Animation,Biography,Comedy,Crime,Documentary,Drama,Family,Fantasy,...,Reality-TV,Romance,Sci-Fi,Short,Sport,Talk-Show,Thriller,War,Western,NaN
2'Live Bre,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,8.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
2Mex,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
50 Cent,4.0,0.0,0.0,1.0,0.0,4.0,0.0,3.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,NaN
A.J. Baime,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,NaN
A.J. Daulerio,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Úrsula Corberó,8.0,0.0,0.0,0.0,0.0,8.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
Úrsula Pruneda,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
Ülkü Duru,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
Þorsteinn Bachmann,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
